In [5]:
import os
import sys
base_dir = os.path.join("/home/jovyan/work/Corona/COVID-19")
sys.path.append(base_dir)
import model as md
import pandas as pd

In [6]:
#NewsExtractor(2)
def createJson(date, day, json=False):
    news_reader = md.NewsReader(data_dir="data", month=date, date="{}-{}".format(date, day), string_filter=True)
    repartition = news_reader.repartition[
        (news_reader.repartition["h1"]=="Az aktív fertőzöttek megyei eloszlása")
        & (news_reader.repartition["date"].dt.day == int(day))
    ]["body"].unique()
    date_of_info = news_reader.repartition[
        (news_reader.repartition["h1"]=="Az aktív fertőzöttek megyei eloszlása")
        & (news_reader.repartition["date"].dt.day == int(day))
    ]["date"].unique()
    print(date_of_info)
    if len(repartition)>0:
        table = "Aktív fertőzöttek száma{}".format(repartition[0].split("Aktív fertőzöttek száma")[1]).split("\n\n\n\n\n")
        df = pd.DataFrame(table, columns=["full"])
        df["full"] = df["full"].str.split("\n\n\n")
        df["len"] = df["full"].apply(lambda cell: len(cell))
        df = df[df["len"]==3].copy()
        df['region'], df['active'], df['death'] = zip(*df["full"])
        df['date'] = "{}-{}".format(date, day)
        df[['date', 'region', 'active', 'death']].to_json('repartition/{}-{}.json'.format(date, day), orient='records')
        return df[['date', 'region', 'active', 'death']]

In [23]:
createJson("2021-02", "21", True)

INFO:root:2021-02-21 09:17:00
INFO:root:https://koronavirus.gov.hu/cikkek/2912-fovel-emelkedett-beazonositott-fertozottek-szama-es-elhunyt-47-beteg
INFO:root:
2912 újabb magyar állampolgárnál mutatták ki az új koronavírus-fertőzést (COVID-19), ezzel 403 023 főre nőtt a hazánkban beazonosított fertőzöttek száma. Elhunyt 47 többségében idős, krónikus beteg, így az elhunytak száma 14 299 főre emelkedett. A gyógyultak száma folyamatosan nő, jelenleg 306 621 fő, az aktív fertőzöttek száma 82 103 fő. 4233 koronavírusos beteget ápolnak kórházban, közülük 366-an vannak lélegeztetőgépen. Eddig 445 535 fő kapott oltást, közülük 190 705 fő már a második oltását is megkapta. Jelenleg is zajlik a regisztrált legidősebbek oltása a kórházi oltópontokon (Pfizer- és Szputnyik vakcinával), a háziorvosok pedig a 60 év alatti krónikus betegek oltását végzik (AstraZeneca-vakcinával).  Zöld utat kapott a hatóságoktól a kínai Sinopharm-vakcina is, így jövő héten azzal is kezdődhet az oltás. Elindult az újrai

[]
